References

- https://www.kaggle.com/code/mpware/vllm-0-7 for the current installation script
- https://www.kaggle.com/code/richolson/ai-math-olympiad-qwen2-5-72b for showing how to submit
- https://www.kaggle.com/code/abdullahmeda/load-72b-awq-model-using-vllm-on-l4-x4

In [1]:
import os
# https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-2/discussion/560682#3113134
os.environ["TRITON_PTXAS_PATH"] = "/usr/local/cuda/bin/ptxas"

In [2]:
import os
import gc
import time
import warnings

import pandas as pd
import polars as pl
import numpy as np

import torch
import kaggle_evaluation.aimo_2_inference_server

pd.set_option('display.max_colwidth', None)
start_time = time.time()
cutoff_time = start_time + (4 * 60 + 45) * 60
cutoff_times = [int(x) for x in np.linspace(cutoff_time, start_time + 60 * 60, 50 + 1)]

In [3]:
from vllm import LLM, SamplingParams

warnings.simplefilter('ignore')

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

if os.getenv('KAGGLE_KERNEL_RUN_TYPE') or os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    llm_model_pth = '/kaggle/input/deepseek-r1/transformers/deepseek-aideepseek-r1-distill-qwen-14b-awq-neody/1'
    # llm_model_pth = '/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b-awq-casperhansen/1'
else:
    llm_model_pth = '/root/volume/KirillR/QwQ-32B-Preview-AWQ'

MAX_NUM_SEQS = 14
MAX_MODEL_LEN = 8192 * 3 // 2

llm = LLM(
    llm_model_pth,
    # dtype="half",                # The data type for the model weights and activations
    max_num_seqs=MAX_NUM_SEQS,   # Maximum number of sequences per iteration. Default is 256
    max_model_len=MAX_MODEL_LEN, # Model context length
    trust_remote_code=True,      # Trust remote code (e.g., from HuggingFace) when downloading the model and tokenizer
    tensor_parallel_size=4,      # The number of GPUs to use for distributed execution with tensor parallelism
    gpu_memory_utilization=0.95, # The ratio (between 0 and 1) of GPU memory to reserve for the model
    seed=2024,
)

INFO 02-18 01:46:32 __init__.py:183] Automatically detected platform cuda.
INFO 02-18 01:47:06 config.py:526] This model supports multiple tasks: {'reward', 'generate', 'embed', 'classify', 'score'}. Defaulting to 'generate'.
INFO 02-18 01:47:09 awq_marlin.py:109] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 02-18 01:47:09 config.py:1383] Defaulting to use mp for distributed inference
WARNING 02-18 01:47:09 config.py:975] MLA is not supported with awq_marlin quantization. Disabling MLA.
INFO 02-18 01:47:09 llm_engine.py:232] Initializing a V0 LLM engine (v0.7.1) with config: model='/kaggle/input/deepseek-r1/transformers/deepseek-aideepseek-r1-distill-qwen-14b-awq-neody/1', speculative_config=None, tokenizer='/kaggle/input/deepseek-r1/transformers/deepseek-aideepseek-r1-distill-qwen-14b-awq-neody/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=to

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


(VllmWorkerProcess pid=351) INFO 02-18 01:48:33 model_runner.py:1116] Loading model weights took 2.3731 GB
INFO 02-18 01:48:33 model_runner.py:1116] Loading model weights took 2.3731 GB
(VllmWorkerProcess pid=359) INFO 02-18 01:48:33 model_runner.py:1116] Loading model weights took 2.3731 GB
(VllmWorkerProcess pid=354) INFO 02-18 01:48:33 model_runner.py:1116] Loading model weights took 2.3731 GB
(VllmWorkerProcess pid=351) (VllmWorkerProcess pid=359) WARNING 02-18 01:48:56 config.py:975] MLA is not supported with awq_marlin quantization. Disabling MLA.
WARNING 02-18 01:48:56 config.py:975] MLA is not supported with awq_marlin quantization. Disabling MLA.
(VllmWorkerProcess pid=351) (VllmWorkerProcess pid=359) WARNING 02-18 01:48:56 config.py:975] MLA is not supported with awq_marlin quantization. Disabling MLA.
(VllmWorkerProcess pid=354) WARNING 02-18 01:48:56 config.py:975] MLA is not supported with awq_marlin quantization. Disabling MLA.
(VllmWorkerProcess pid=351) WARNING 02-18 01

Capturing CUDA graph shapes:  80%|████████  | 4/5 [00:04<00:00,  1.00it/s]

(VllmWorkerProcess pid=351) INFO 02-18 01:49:07 model_runner.py:1563] Graph capturing finished in 6 secs, took 0.09 GiB


Capturing CUDA graph shapes: 100%|██████████| 5/5 [00:05<00:00,  1.10s/it]

(VllmWorkerProcess pid=359) INFO 02-18 01:49:07 model_runner.py:1563] Graph capturing finished in 6 secs, took 0.09 GiB
(VllmWorkerProcess pid=354) INFO 02-18 01:49:07 model_runner.py:1563] Graph capturing finished in 6 secs, took 0.09 GiB
INFO 02-18 01:49:07 model_runner.py:1563] Graph capturing finished in 6 secs, took 0.09 GiB
INFO 02-18 01:49:07 llm_engine.py:429] init engine (profile, create kv cache, warmup model) took 33.05 seconds


In [4]:
import vllm
print(vllm.__version__)

0.7.1


In [5]:
tokenizer = llm.get_tokenizer()

In [6]:
import re
import keyword


def extract_boxed_text(text):
    pattern = r'oxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return ""
    for match in matches[::-1]:
        if match != "":
            return match
    return ""


from collections import Counter
import random
def select_answer(answers):
    counter = Counter()
    for answer in answers:
        try:
            if int(answer) == float(answer):
                counter[int(answer)] += 1 + random.random() / 1_000
        except:
            pass
    if not counter:
        return 210
    _, answer = sorted([(v,k) for k,v in counter.items()], reverse=True)[0]
    return answer%1000

In [7]:
def batch_message_generate(list_of_messages) -> list[list[dict]]:
    max_tokens = MAX_MODEL_LEN
    if time.time() > cutoff_times[-1]:
        print("Speedrun")
        max_tokens = 2 * MAX_MODEL_LEN // 3

    sampling_params = SamplingParams(
        temperature=0.7,              # randomness of the sampling
        min_p=0.01,
        skip_special_tokens=True,     # Whether to skip special tokens in the output
        max_tokens=max_tokens,
        stop=["</think>"]
    )
    
    list_of_texts = [
        tokenizer.apply_chat_template(
            conversation=messages,
            tokenize=False,
            add_generation_prompt=True
        )
        for messages in list_of_messages
    ]

    request_output = llm.generate(
        prompts=list_of_texts,
        sampling_params=sampling_params,
    )
    
    print([len(single_request_output.outputs[0].token_ids) for single_request_output in request_output])

    sort_keys_and_list_of_messages = []

    for messages, single_request_output in zip(list_of_messages, request_output):
        # print()
        # print(single_request_output.outputs[0].text)
        # print()
        messages.append({'role': 'assistant', 'content': single_request_output.outputs[0].text})

        sort_keys_and_list_of_messages.append(
            (
                len(single_request_output.outputs[0].token_ids),
                messages
            )
        )

    print([sort_key for sort_key, _ in sort_keys_and_list_of_messages])
    sort_keys_and_list_of_messages.sort(key=lambda sort_key_and_messages: sort_key_and_messages[0])
    print([sort_key for sort_key, _ in sort_keys_and_list_of_messages])

    list_of_messages = [messages for _, messages in sort_keys_and_list_of_messages]
    
    return list_of_messages

In [8]:
def batch_message_filter(list_of_messages) -> tuple[list[list[dict]], list[str]]:
    extracted_answers = []
    list_of_messages_to_keep = []
    for messages in list_of_messages:
        answer = extract_boxed_text(messages[-1]['content'])
        if answer:
            extracted_answers.append(answer)
        else:
            list_of_messages_to_keep.append(messages)
    return list_of_messages_to_keep, extracted_answers

In [9]:
thought_prefix_english = """<think>Math Experts categorize question into one of the following types: "Number Theory, Combinatorial Mathematics, Algebraic Inequalities, Geometric Constructions, Functional Equations, or Others."
"""

problem_cate = """
If the question is about Number Theory, please consider:
Understand the problem and try simple examples; use basic tools (prime factorization, divisibility, modular arithmetic, Euclid’s algorithm); look for invariants or recursive patterns; simplify or transform conditions via induction.

If the question is about Combinatorial Mathematics, please consider:
Apply counting principles (permutations, combinations, inclusion-exclusion); use double counting or bijections; seek extremal values or invariants; consider recurrence relations or generating functions for an algebraic reformulation.

If the question is about Algebraic Inequalities, please consider:
Use classical inequalities (AM-GM, Cauchy-Schwarz, etc.); homogenize or normalize the expression; perform variable substitution or smoothing techniques; construct auxiliary functions to exploit monotonicity or convexity.

If the question is about Geometric Constructions, please consider:
Draw an accurate diagram and add necessary auxiliary lines or circles; analyze angles and proportions; employ geometric transformations (inversion, rotation, reflection) or use coordinate methods when appropriate.

If the question is about Functional Equations, please consider:
Substitute special values (like 0, 1, or equal variables) to simplify the equation; examine symmetry, periodicity, or monotonicity; prove uniqueness or construct all possible solutions through iterative substitution.

If the question is about Others, please consider this way:
First classify the problem and analyze its structure for hidden relationships or symmetry; combine methods from various fields as needed; stay flexible and adjust your approach based on specific features.
"""



though_prefix_chinese = """<think>数学专家会对问题进行分类：识别为‘数论、组合数学、代数不等式、几何构造、函数方程、其它’类型中的一个类型。 

"""
problem_cate_china = """
如果题目是关于数论，请参考：
理解题意并尝试构造简单例子；运用基本工具（素数分解、整除性质、模算术、欧几里得算法）；寻找不变量或递归结构；利用归纳法或转换简化条件。

如果题目是关于组合数学，请参考：
运用排列组合、容斥原理等计数技巧；尝试双重计数或构造双射；寻找极值条件或不变量；考虑递归关系或生成函数将问题转化为代数形式。

如果题目是关于代数不等式，请参考：
使用经典不等式工具（如 AM-GM、不等式、Cauchy-Schwarz 等）；对表达式进行归一化或同次化；采用变量替换或平滑化方法；构造辅助函数利用单调性或凸凹性证明不等式。

如果题目是关于几何构造，请参考：
仔细绘图并标记关键点，添加必要的辅助线或圆；分析角度和比例关系；使用几何变换（反演、旋转、平移）或坐标方法进行解析证明。

如果题目是关于泛函方程，请参考：
取特殊值（如 0、1 或令变量相等）简化方程；分析函数的对称性、周期性和单调性；证明解的唯一性或构造所有满足条件的解；利用迭代代入等技巧逐步缩小解的范围。

如果题目属于其他类型，请参考：
先判断题目所属领域，分析题目结构，寻找隐含的逻辑关系或对称性；必要时综合运用数论、组合、几何等多种方法；灵活调整解题策略，适时归纳总结或应用数学归纳法。
"""


def create_starter_messages(question, index):
    options = []
    for _ in range(12):
        options.append(
            [
                {"role": "system", "content":thought_prefix_english +  "You are a the most powerful math expert. Please solve the problems with deep resoning. You are careful and always recheck your conduction. You will never give answer directly until you have enough confidence. You should think step-by-step. Return final answer within \\boxed{}, after taking modulo 1000."},
                {"role": "user", "content": question+problem_cate },
            ]
        )
    for _ in range(1):    
        options.append(
            [
                {"role": "system", "content": thought_prefix_english +  "You are a helpful and harmless math expert. You should think step-by-step and you are good at reverse thinking to recheck your answer and fix all possible mistakes. After you get your final answer, take modulo 1000, and return the final answer within \\boxed{}."},
                {"role": "user", "content": question+problem_cate},
            ],
        )
    options.append(
        [
            {"role": "system", "content": though_prefix_chinese + "你是数学解题专家。请先仔细阅读题目，确保自己理解了题意和考点，再通过深度逐步推理来完整正确地解答问题，并把最终答案对1000取余数，放置于\\boxed{}中。"},
            {"role": "user", "content": question+problem_cate_china},
        ],
    )
    return options[index%len(options)]

In [10]:
def predict_for_question(question: str) -> int:
    import os

    selected_questions_only = True
    # selected_questions_only = False
    if selected_questions_only and not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        # if "Triangle" not in question:
        #     return 210
        if "Triangle" not in question and "delightful" not in question and "George" not in question:
            return 210

    if time.time() > cutoff_time:
        return 210
    
    print(question)

    num_seqs = MAX_NUM_SEQS
    if time.time() > cutoff_times[-1]:
        num_seqs = 2 * MAX_NUM_SEQS // 3
    
    list_of_messages = [create_starter_messages(question, index) for index in range(num_seqs)]

    all_extracted_answers = []
    for _ in range(1):
        list_of_messages = batch_message_generate(list_of_messages)
        
        if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
            df = pd.DataFrame(
                {
                    "question": [question] * len(list_of_messages),
                    "message": [messages[-1]["content"] for messages in list_of_messages],
                }
            )
            df.to_csv(f"{str(int(time.time() - start_time)).zfill(5)}.csv", index=False)
        
        list_of_messages, extracted_answers = batch_message_filter(list_of_messages)
        all_extracted_answers.extend(extracted_answers)
    
    print(all_extracted_answers)
    answer = select_answer(all_extracted_answers)
    print(answer)

    print("\n\n")
    cutoff_times.pop()
    return answer

In [11]:
# Replace this function with your inference code.
# The function should return a single integer between 0 and 999, inclusive.
# Each prediction (except the very first) must be returned within 30 minutes of the question being provided.
def predict(id_: pl.DataFrame, question: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
    id_ = id_.item(0)
    print("------")
    print(id_)
    
    question = question.item(0)
    answer = predict_for_question(question)
    print(question)
    print("------\n\n\n")
    return pl.DataFrame({'id': id_, 'answer': answer})

In [12]:
# predict_for_question("Triangle $ABC$ has side length $AB = 120$ and circumradius $R = 100$. Let $D$ be the foot of the perpendicular from $C$ to the line $AB$. What is the greatest possible length of segment $CD$?")

In [13]:
pd.read_csv(
    '/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv'
).drop('answer', axis=1).to_csv('reference.csv', index=False)

In [14]:
inference_server = kaggle_evaluation.aimo_2_inference_server.AIMO2InferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
#             '/kaggle/input/ai-mathematical-olympiad-progress-prize-2/test.csv',
            'reference.csv',
        )
    )

------
1fce4b
Find the three-digit number $n$ such that writing any other three-digit number $10^{2024}$ times in a row and $10^{2024}+2$ times in a row results in two numbers divisible by $n$.
------



------
480182
Let $ABC$ be a triangle with $BC=108$, $CA=126$, and $AB=39$. Point $X$ lies on segment $AC$ such that $BX$ bisects $\angle CBA$. Let $\omega$ be the circumcircle of triangle $ABX$. Let $Y$ be a point on $\omega$ different from $X$ such that $CX=CY$. Line $XY$ meets $BC$ at $E$. The length of the segment $BE$ can be written as $\frac{m}{n}$, where $m$ and $n$ are coprime positive integers. Find $m+n$.
------



------
bbd91e
Alice writes all positive integers from $1$ to $n$ on the board for some positive integer $n \geq 11$. Bob then erases ten of them. The mean of the remaining numbers is $3000/37$. The sum of the numbers Bob erased is $S$. What is the remainder when $n \times S$ is divided by $997$?
------



------
71beb6
For a positive integer $n$, let $S(n)$ denote 

Processed prompts: 100%|██████████| 14/14 [05:35<00:00, 23.93s/it, est. speed input: 20.16 toks/s, output: 251.75 toks/s]


[9387, 6659, 5017, 7232, 3932, 4327, 6415, 5908, 7632, 10207, 4441, 6682, 4475, 2042]
[9387, 6659, 5017, 7232, 3932, 4327, 6415, 5908, 7632, 10207, 4441, 6682, 4475, 2042]
[2042, 3932, 4327, 4441, 4475, 5017, 5908, 6415, 6659, 6682, 7232, 7632, 9387, 10207]
['180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '160', '180', '180']
180



Triangle $ABC$ has side length $AB = 120$ and circumradius $R = 100$. Let $D$ be the foot of the perpendicular from $C$ to the line $AB$. What is the greatest possible length of segment $CD$?
------



------
349493
We call a sequence $a_1, a_2, \ldots$ of non-negative integers \textit{delightful} if there exists a positive integer $N$ such that for all $n > N$, $a_n = 0$, and for all $i \geq 1$, $a_i$ counts the number of multiples of $i$ in $a_1, a_2, \ldots, a_N$. How many delightful sequences of non-negative integers are there?


Processed prompts: 100%|██████████| 14/14 [07:59<00:00, 34.23s/it, est. speed input: 15.50 toks/s, output: 334.14 toks/s]


[11760, 11760, 11760, 11760, 11760, 11760, 11760, 11760, 11760, 11760, 11760, 10655, 11767, 8360]
[11760, 11760, 11760, 11760, 11760, 11760, 11760, 11760, 11760, 11760, 11760, 10655, 11767, 8360]
[8360, 10655, 11760, 11760, 11760, 11760, 11760, 11760, 11760, 11760, 11760, 11760, 11760, 11767]
['1', '1']
1



We call a sequence $a_1, a_2, \ldots$ of non-negative integers \textit{delightful} if there exists a positive integer $N$ such that for all $n > N$, $a_n = 0$, and for all $i \geq 1$, $a_i$ counts the number of multiples of $i$ in $a_1, a_2, \ldots, a_N$. How many delightful sequences of non-negative integers are there?
------



------
88c219
For positive integers $x_1,\ldots, x_n$ define $G(x_1, \ldots, x_n)$ to be the sum of their $\frac{n(n-1)}{2}$ pairwise greatest common divisors. We say that an integer $n \geq 2$ is \emph{artificial} if there exist $n$ different positive integers $a_1, ..., a_n$ such that 
\[a_1 + \cdots + a_n = G(a_1, \ldots, a_n) +1.\]
Find the sum of all 

Processed prompts: 100%|██████████| 14/14 [07:46<00:00, 33.32s/it, est. speed input: 15.35 toks/s, output: 322.03 toks/s]

[11779, 7069, 10229, 11779, 11779, 11779, 11779, 11779, 9071, 11779, 7724, 11779, 11786, 10115]
[11779, 7069, 10229, 11779, 11779, 11779, 11779, 11779, 9071, 11779, 7724, 11779, 11786, 10115]
[7069, 7724, 9071, 10115, 10229, 11779, 11779, 11779, 11779, 11779, 11779, 11779, 11779, 11786]
['750', '250', '0', '0']
0



Fred and George take part in a tennis tournament with $4046$ other players. In each round, the players are paired into $2024$ matches. How many ways are there to arrange the first round such that Fred and George do not have to play each other? (Two arrangements for the first round are \textit{different} if there is a player with a different opponent in the two arrangements.)
------



------
a1d40b
The Fibonacci numbers are defined as follows: $F_0 = 0$, $F_1 = 1$, and $F_{n+1} = F_n + F_{n-1}$ for $n \geq 1$. There are $N$ positive integers $n$ strictly less than $10^{101}$ such that $n^2 + (n+1)^2$ is a multiple of 5 but $F_{n-1}^2 + F_n^2$ is not. How many prime factors 